In [ ]:
import os
import sys
import json
import pandas as pd
from datetime import datetime
from re import sub

# Project imports
sys.path.append(os.getcwd())
# Finance utils
from src.py.analysis.yahoo.stocks.finance_df_utils import load_stock_csv, df_add_data, get_figure, filter_df_by_date, add_vline_annotation, save_fig, get_safe_filename, get_grouped_df
# Events
from src.py.analysis.events import events
# CPI
import importlib
sys.path.append(os.getcwd())
cpi_adjust = importlib.import_module("src.py.scraping.world-bank.cpi_adjust")
cpi_adjust.initialize_cpi(date_cutoff="2018-08-01", jagged=False) # jagged=True)

In [ ]:
path_stocks_csv_root = "data/scraped/yahoo/stocks/csv"
path_companies_index = "data/scraped/yahoo/sectors/index_stocks.json"
path_output_root = "data/analysis/yahoo/stocks"

if os.path.exists(path_output_root) is False:
	os.makedirs(path_output_root)


In [ ]:
# Load data
df_msft = load_stock_csv(os.path.join(path_stocks_csv_root, "MSFT.csv"))

# Filter after 2018-10-01 (leave some for SMA)
df_msft = filter_df_by_date(df_msft, start_date="2018-10-01")

# Adjust for inflation
df_msft = cpi_adjust.adjust_for_inflation(df_msft, "USA", columns=["Open", "High", "Low", "Close", "Adj Close"])

# Add data
df_msft = df_add_data(df_msft)

# Filter after 2019-01-01
df_msft = filter_df_by_date(df_msft, start_date="2019-01-01")

# Resample to weekly
df_msft = df_msft.resample('W').mean()

# Show tail
df_msft.tail()

In [ ]:
# Alternative using Volume_value as Volume (abs(Open+Close) / 2 * Volume)

# Load data
df_msft = load_stock_csv(os.path.join(path_stocks_csv_root, "MSFT.csv"))

# Filter after 2018-10-01 (leave some for SMA)
df_msft = filter_df_by_date(df_msft, start_date="2018-10-01")

# Add Volume_value column
df_msft["Volume_value"] = (abs(df_msft["Open"] + df_msft["Close"]) / 2) * df_msft["Volume"]
# Replace Volume with Volume_value and drop Volume_value
df_msft["Volume"] = df_msft["Volume_value"]
df_msft = df_msft.drop(columns=["Volume_value"])

# Adjust for inflation - now with Volume
df_msft = cpi_adjust.adjust_for_inflation(df_msft, "USA", columns=["Open", "High", "Low", "Close", "Adj Close", "Volume"])

# Add data
df_msft = df_add_data(df_msft)

# Filter after 2019-01-01
df_msft = filter_df_by_date(df_msft, start_date="2019-01-01")

# Resample to weekly
df_msft = df_msft.resample('W').mean()

# Show tail
df_msft.tail()

In [ ]:
# Sample:

options = {
 "w": 1280,
 "h": 720,
 # "traces": ["candlestick"],
 "traces": [],
 "margin": {"r": 0, "t": 60, "b": 0, "l": 0},
 "labels": ["Price ($)", "Volume", "MACD"],
}

# Get figure
fig = get_figure(df_msft, "Microsoft (MSFT) stock price", options)

# # Add annotation
# event_covid_crash = {
#   "date": datetime(2020, 2, 20),
#   # "annotation": "COVID-19 market crash"
# 	"annotation": "MC"
# }
# add_vline_annotation(fig, event_covid_crash)

# # Add annotation
# event_covid_crash_end = {
#   "date": datetime(2020, 4, 7),
#   "annotation": "MC end"
# }
# add_vline_annotation(fig, event_covid_crash_end)

# for event in events:
# 	add_vline_annotation(fig, event)


# Show figure
fig.show()

In [ ]:
# Load companies
companies = json.load(open(path_companies_index, "r"))
companies = {company["symbol"]: company for company in companies}
print(f"Loaded index with {len(companies)} companies.")

In [ ]:
def add_index_values(df: pd.DataFrame, date: str) -> pd.DataFrame:
	index_value = df.loc[date, "Open"]
	df["Open_I"] = df["Open"] / index_value * 100  # type: ignore
	index_value = df.loc[date, "High"]
	df["High_I"] = df["High"] / index_value * 100  # type: ignore
	index_value = df.loc[date, "Low"]
	df["Low_I"] = df["Low"] / index_value * 100  # type: ignore
	index_value = df.loc[date, "Close"]
	df["Close_I"] = df["Close"] / index_value * 100  # type: ignore
	index_value = df.loc[date, "Adj Close"]
	df["Adj_Close_I"] = df["Adj Close"] / index_value * 100  # type: ignore
	# index_value = df.loc[date, "Volume"]
	# df["Volume_I"] = df["Volume"] / index_value * 100  # type: ignore
	# NOTE: this approach can scramble the the values because of relative scaling
	# 		  as a consequence, the values like High_I can be lower than Low_I
	# Invert the values where High < Low
	# df["temp"] = df["High_I"]
	# df.loc[df["High_I"] < df["Low_I"],
	#        "High_I"] = df.loc[df["High_I"] < df["Low_I"], "Low_I"]
	# df.loc[df["High_I"] < df["Low_I"],
	#        "Low_I"] = df.loc[df["High_I"] < df["Low_I"], "temp"]
	# # Switch the Close_I, Open_I for all rows where their order is different from Close, Open
	# rows_og = df["Close"] < df["Open"]
	# rows_i = df["Close_I"] < df["Open_I"]
	# mask = rows_og != rows_i
	# count_pre = mask.sum()
	# df.loc[mask, "temp"] = df.loc[mask, "Close_I"]
	# df.loc[mask, "Close_I"] = df.loc[mask, "Open_I"]
	# df.loc[mask, "Open_I"] = df.loc[mask, "temp"]
	# rows_og = df["Close"] < df["Open"]
	# rows_i = df["Close_I"] < df["Open_I"]
	# mask = rows_og != rows_i
	# count_post = mask.sum()
	# # Drop temp column
	# df.drop(columns=["temp"], inplace=True)
	# NOTE: a more complete approach would be to scale Open_I, High_I, and Low_I proportionally to Open, High, and Low using Close_I as a base
	#       calculated from the relative scaling of Close_I and Close - each daily measurement / candle should retain the same proportions and order
	return df

# Replace column values of Open with Open_I, High with High_I, Low with Low_I, Close with Close_I, Adj Close with Adj_Close_I, Volume with Volume_I
def replace_by_index_value(df: pd.DataFrame) -> pd.DataFrame:
	if "Open_I" in df.columns:
		# drop original columns
		df = df.drop(
		    columns=["Open", "High", "Low", "Close", "Adj Close"])#, "Volume"])
		# rename columns
		df = df.rename(
		    columns={
		        "Open_I": "Open",
		        "High_I": "High",
		        "Low_I": "Low",
		        "Close_I": "Close",
		        "Adj_Close_I": "Adj Close",
		        # "Volume_I": "Volume"
		    })
	# put the columns in the original order
	df = df[["Open", "High", "Low", "Close", "Adj Close", "Volume"]]
	return df

# Load data
df_msft = load_stock_csv(os.path.join(path_stocks_csv_root, "MSFT.csv"))
# Filter after 2018-10-01 (leave some for SMA)
df_msft = filter_df_by_date(df_msft, start_date="2018-08-01")
# Adjust for inflation
df_msft = cpi_adjust.adjust_for_inflation(df_msft, "USA", columns=["Open", "High", "Low", "Close", "Adj Close"])
# Filter after 2019-01-01
df_msft = filter_df_by_date(df_msft, start_date="2019-01-01")
# Add index values
df_msft = add_index_values(df_msft, "2019-01-02")
# Replace by index value
df_msft = replace_by_index_value(df_msft)
# df_scaled.head()
# show df from 2019-01-01 to 2019-01-05
df_msft.loc["2019-01-01":"2019-01-05"]

In [ ]:
# # NOTE: ORIGINAL

# # Load all CSVs
# dfs_dict = {}
# dfs_fails = {}
# print(f"Loading {len(companies)} company stocks CSVs.")
# print("")
# for i, (ticker, company) in enumerate(companies.items()):
# 	print(f"{i+1}/{len(companies)} [{ticker}]     ", end="\r")
# 	try:
# 		# Load CSV
# 		df = load_stock_csv(os.path.join(path_stocks_csv_root, f"{ticker}.csv"))
# 		# Check if data is within range (first date < 2018-10-01)
# 		if df.index[0] > datetime(2018, 8, 1):
# 			raise Exception("Data starts after 2018-08-01.")
# 		# Check if data is within range (last date > 2023-12-01)
# 		if df.index[-1] < datetime(2023, 12, 1):
# 			raise Exception("Data ends before 2023-12-01.")
# 		# Plotting interval is 2019-01-01 to 2023-12-26
# 		# start_date is set to 2018-10-01 to leave some for window
# 		df = filter_df_by_date(df, start_date="2018-08-01", end_date="2023-12-26")
# 		# Adjust for CPI
# 		df = cpi_adjust.adjust_for_inflation(df, "USA", columns=["Open", "High", "Low", "Close", "Adj Close"])
# 		# Remove all rows with NaN values
# 		df = df.dropna()
# 		# Add index values
# 		df = add_index_values(df, "2019-01-02") # 2019-01-01 is a holiday (no trading)
# 		# # Replace by index value
# 		df = replace_by_index_value(df)
# 		# Add to dict
# 		dfs_dict[ticker] = df
# 	except Exception as e:
# 		dfs_fails[ticker] = e
# print("")
# print(f"Loaded {len(dfs_dict)} company stocks CSVs.")
# print(f"{len(dfs_fails)} failed to load or do not meet the criteria.")


In [ ]:
# NOTE: ADJUSTED - using Volume_value

# Load all CSVs
dfs_dict = {}
dfs_fails = {}
print(f"Loading {len(companies)} company stocks CSVs.")
print("")
for i, (ticker, company) in enumerate(companies.items()):
	print(f"{i+1}/{len(companies)} [{ticker}]     ", end="\r")
	try:
		# Load CSV
		df = load_stock_csv(os.path.join(path_stocks_csv_root, f"{ticker}.csv"))
		# Check if data is within range (first date < 2018-10-01)
		if df.index[0] > datetime(2018, 8, 1):
			raise Exception("Data starts after 2018-08-01.")
		# Check if data is within range (last date > 2023-12-01)
		if df.index[-1] < datetime(2023, 12, 1):
			raise Exception("Data ends before 2023-12-01.")
		# Plotting interval is 2019-01-01 to 2023-12-26
		# start_date is set to 2018-10-01 to leave some for window
		df = filter_df_by_date(df, start_date="2018-08-01", end_date="2023-12-26")
		# Add Volume_value column
		df["Volume_value"] = (abs(df["Open"] + df["Close"]) / 2) * df["Volume"]
		# Replace Volume with Volume_value and drop Volume_value
		df["Volume"] = df["Volume_value"]
		df = df.drop(columns=["Volume_value"])
		# Adjust for CPI
		df = cpi_adjust.adjust_for_inflation(df, "USA", columns=["Open", "High", "Low", "Close", "Adj Close", "Volume"])
		# Remove all rows with NaN values
		df = df.dropna()
		# Add index values
		df = add_index_values(df, "2019-01-02") # 2019-01-01 is a holiday (no trading)
		# # Replace by index value
		df = replace_by_index_value(df)
		# Add to dict
		dfs_dict[ticker] = df
	except Exception as e:
		dfs_fails[ticker] = e
print("")
print(f"Loaded {len(dfs_dict)} company stocks CSVs.")
print(f"{len(dfs_fails)} failed to load or do not meet the criteria.")


In [ ]:
# Print fails grouped and sorted by error
print(f"Failed to load {len(dfs_fails)} company stocks CSVs.")
print("")
dfs_fails_grouped = { str(e): [] for e in set(dfs_fails.values()) }
for ticker, e in dfs_fails.items():
	dfs_fails_grouped[str(e)].append(ticker)

for e, tickers in sorted(dfs_fails_grouped.items(), key=lambda x: len(x[1]), reverse=True):
	print(f"{len(tickers)} companies: {e}")
	print(f"{', '.join(tickers[:10])}...")
	print("")


In [ ]:
df_msft = dfs_dict["MSFT"]
df_msft.loc["2019-01-01":].head()

In [ ]:
# Filter companies (multiple filters)

# Remove companies without profile
companies = {k: v for k, v in companies.items() if "profile" in v}
print(f"{len(companies)} companies remaining after removing companies without profile.")

# Remove companies without sector (empty string should be removed)
companies = {k: v for k, v in companies.items() if "sector" in v["profile"] and v["profile"]["sector"].strip() != ""}
print(f"{len(companies)} companies remaining after removing companies without sector.")

# Remove companies without industry (empty string should be removed)
companies = {k: v for k, v in companies.items() if "industry" in v["profile"] and v["profile"]["industry"].strip() != ""}
print(f"{len(companies)} companies remaining after removing companies without industry.")

# Filter companies that are not in the dfs_dict
for ticker in list(companies.keys()):
	if ticker not in dfs_dict:
		del companies[ticker]
# Filter dfs_dict that are not in the companies
for ticker in list(dfs_dict.keys()):
	if ticker not in companies:
		del dfs_dict[ticker]
print(f"{len(companies)} companies remaining after removing companies with data that doesn't fit criteria.")
print(f"{len(dfs_dict)} (dfs_dict) == {len(companies)} (companies) = {len(dfs_dict) == len(companies)}")

In [ ]:
sectors = {}
industries = {}
industries_to_sectors = {}
for ticker, company in companies.items():
	sector = company["profile"]["sector"]
	industry = company["profile"]["industry"]
	if sector not in sectors:
		sectors[sector] = []
	if industry not in industries:
		industries[industry] = []
	if industry not in industries_to_sectors:
		industries_to_sectors[industry] = sector
	sectors[sector].append(ticker)
	industries[industry].append(ticker)

print(f"Fetched {len(sectors)} sectors and {len(industries)} industries from {len(companies)} companies.")
print(f"Sample: Industry 'Software—Application' -> sector '{industries_to_sectors['Software—Application']}'")

In [ ]:
print(f"Sum all companies: {sum([len(tickers) for tickers in sectors.values()])}")
# Print sorted sectors by number of companies
for sector, tickers in sorted(sectors.items(), key=lambda x: len(x[1]), reverse=True):
	print(f"- '{sector}': {len(tickers)} companies")

In [ ]:
# Print sorted industries by number of companies
for industry, tickers in sorted(industries.items(), key=lambda x: len(x[1]), reverse=True):
	print(f"- '{industry}' (sector '{industries_to_sectors[industry]}'): {len(tickers)} companies")

In [ ]:
# Add columns Symbol, Sector, Industry to start of each dataframe and concatenate them into one big dataframe
for symbol, df in dfs_dict.items():
	if "Symbol" in df.columns and df.columns.tolist()[0] == "Symbol": # ensure idempotence, manually reload if needed
		continue
	company = companies[symbol]
	df["Symbol"] = symbol
	df["Sector"] = company["profile"]["sector"]
	df["Industry"] = company["profile"]["industry"]
	# Put Symbol, Sector, Industry columns at the start
	cols = df.columns.tolist()
	cols = cols[-3:] + cols[:-3]
	df = df[cols]
	dfs_dict[symbol] = df

df_all = pd.concat(dfs_dict.values())
df_all.head()

In [ ]:
# Print number of rows and columns
print(f"Number of rows: {df_all.shape[0]}")
print(f"Number of columns: {df_all.shape[1]}")

# Print size in memory
print(f"Size in memory: {df_all.memory_usage().sum() / 1024**2:.2f} MB")

In [ ]:
options = {
 	"w": 1280,
 	"h": 720,
 # "traces": ["candlestick"],
 	"traces": [],
	"color_changes": False,
 	"margin": {"r": 0, "t": 60, "b": 0, "l": 0},
 	"labels": ["Index values", "Volume", "MACD"],
}

In [ ]:
# Generate dfs for all sectors
dfs_sectors = {}
for i, (sector, tickers) in enumerate(sectors.items()):
	print(f"{i+1}/{len(sectors)} Processing sector '{sector}' ({len(tickers)} companies)                                     ", end="\r")
	df_sector = get_grouped_df(df_all, "Sector", sector, start_date="2019-01-01")
	
	# Resample to weekly
	# df_sector = df_sector.resample('W').mean()
	volume = df_sector["Volume"].resample("W").sum()
	volume_sma = df_sector["SMA_volume"].resample("W").sum()
	df_sector = df_sector.resample("W").mean()
	df_sector["Volume"] = volume
	df_sector["SMA_volume"] = volume_sma
	
	# Add to dict
	dfs_sectors[sector] = df_sector

df_sector_technology = dfs_sectors["Technology"]
# df_sector_technology.head(3)
# df_sector_technology.tail(3)
df_sector_technology.head(10)
	

In [ ]:
# Generate dfs for all industries
dfs_industries = {}
for i, (industry, tickers) in enumerate(industries.items()):
	print(f"{i+1}/{len(industries)} Processing industry '{industry}' from sector '{industries_to_sectors[industry]}' ({len(tickers)} companies)                                     ", end="\r")
	df_industry = get_grouped_df(df_all, "Industry", industry, start_date="2019-01-01")
	
	# Resample to weekly
	# df_industry = df_industry.resample('W').mean()
	volume = df_industry["Volume"].resample("W").sum()
	volume_sma = df_industry["SMA_volume"].resample("W").sum()
	df_industry = df_industry.resample("W").mean()
	df_industry["Volume"] = volume
	df_industry["SMA_volume"] = volume_sma

	# Add to dict
	dfs_industries[industry] = df_industry

df_industry_software_application = dfs_industries["Software—Application"]
# df_industry_software_application.head(3)
# df_industry_software_application.tail(3)
df_industry_software_application.head(10)

In [ ]:
# Generate plots for all sectors
for i, (sector, df_sector) in enumerate(dfs_sectors.items()):
	print(f"{i+1}/{len(sectors)} Generating plot for sector '{sector}' ({len(sectors[sector])} companies)                                     ", end="\r")
	# Get figure 
	fig = get_figure(df_sector, f"Market performance of {sector} sector ({len(sectors[sector])} companies)", options=options)
	# Add annotation
	for event in events:
		add_vline_annotation(fig, event, textangle=-20)
	# Save figure
	save_fig(fig, os.path.join(path_output_root, get_safe_filename(f"sector-{sector}.png")), scale=3)

In [ ]:
# Generate plots for all industries
for i, (industry, df_industry) in enumerate(dfs_industries.items()):
	print(f"{i+1}/{len(industries)} Generating plot for industry '{industry}' from sector '{industries_to_sectors[industry]}' ({len(industries[industry])} companies)                                     ", end="\r")
	# Get figure
	fig = get_figure(df_industry, f"Market performance of '{industry}' industry ({len(industries[industry])} companies; Sector '{industries_to_sectors[industry]}')", options=options)
	# Add annotation
	for event in events:
		add_vline_annotation(fig, event, textangle=-20)
	# Save figure
	save_fig(fig, os.path.join(path_output_root, get_safe_filename(f"sector-{industries_to_sectors[industry]}-industry-{industry}.png")), scale=3)

In [ ]:
# Generate df for df_all
df_all_grouped = get_grouped_df(df_all, start_date="2019-01-01")

# Resample to weekly
# df_all_grouped = df_all_grouped.resample('W').mean()
volume = df_all_grouped["Volume"].resample("W").sum()
volume_sma = df_all_grouped["SMA_volume"].resample("W").sum()
df_all_grouped = df_all_grouped.resample("W").mean()
df_all_grouped["Volume"] = volume
df_all_grouped["SMA_volume"] = volume_sma

df_all_grouped.head(10)

In [ ]:
options_alt = {
	"w": 1280,
	"h": 720,
	"traces": [],
	"color_changes": False,
	"margin": {"r": 0, "t": 60, "b": 0, "l": 0},
	"labels": ["Index values", "Volume", "MACD"],
}

In [ ]:
# Get figure
fig = get_figure(df_all_grouped, f"Market performance (all {len(companies)} companies)", options=options_alt)
# Add annotations
for event in events:
	add_vline_annotation(fig, event)
# Save figure
save_fig(fig, os.path.join(path_output_root, get_safe_filename(f"entire-market.png")), scale=3)

fig.show()